In [171]:
import ghmm
from collections import OrderedDict
import pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"

In [172]:
NMERS = 3
NSTATES = 4**NMERS
MULTIVARIATE = True

In [173]:
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params_{0}mer.pickle".format(NMERS),
    "ncores": 4
}

# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params_{0}mer.pickle".format(NMERS),
#     "ncores": 62
# }

In [174]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = OrderedDict(HMM_PARAMS)
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS.keys() == ALL_KMERS

# Train Model 

In [175]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [176]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [177]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (20/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (29/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [178]:
mk_transmat = mk_transmat2

In [179]:
F = ghmm.Float()  # emission domain of this model

def mk_model_multivariate():
    # example code for a continuous HMM with gaussian emissions

    A = mk_transmat(NMERS)
    B = [ [ 
            [float(df[['mean']].mean()), float(df[['stdv']].mean())], #mu1, mu2
            [x for x in chain(*df[['mean', 'stdv']].cov().values.tolist())], #covariance matrix
        ] for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.MultivariateGaussianDistribution(F), A, B, pi)
    return model

In [180]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = [ [float(df[['mean']].mean()), float(df[['stdv']].mean())] #mu1, stdv
            for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [181]:
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [182]:
model = mk_model()
s = str(model)
print(s)


HMM Overview:
Number of states: 64
maximum Number of mixture components: 1
Number of dimensions: 2

State number 0:
Initial probability: 0.015625
Number of mixture components: 1

  Emission number 0:
    emission type: 6
    emission weight: 1.0
    mean: [70.16634359689925, 1.4638138587468257]
    covariance matrix: [14.74402884794052, 0.3425982497214579, 0.3425982497214579, 0.13456977667107364]
    inverse of covariance matrix: [0.07208861758745962, -0.18352883404622602, -0.18352883404622602, 7.898331138020882]
    determinant of covariance matrix: 1.86672710859
    cholesky decomposition of covariance matrix: [14.74402884794052, 0.3425982497214579, 0.3425982497214579, 0.13456977667107364]
    fix: 0

  Class : 0
    Outgoing transitions:
      transition to state 0 with probability = 0.025
      transition to state 1 with probability = 0.025
      transition to state 2 with probability = 0.025
      transition to state 3 with probability = 0.025
      transition to state 4 with pro

In [183]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [184]:
def predict(mixed):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    if MULTIVARIATE: 
        emissions = [x for x in chain(*mixed)]    
    else: 
        emissions = [x[0] for x in mixed]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [185]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [186]:
predict(seq)

'CGCTTTCTCT'

# Validate Model 

In [187]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline


In [188]:
assert os.path.isfile(args["events"])

In [189]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>mmusMT_PCR1']
GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACACAAAGGTTTGGTCCTGGCCTTATAATTAATTA


In [190]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [191]:
prepare_filemap(file_data)

### Make Alignment

In [192]:
fastq_file = "{0}.fastq".format(args["out_basename"])
mk_fastq(fastq_file, file_data)

In [193]:
sam_file = "{0}.sam".format(args["out_basename"])
graphmap(ref_file, fastq_file, sam_file, args["ncores"])

[Index 16:04:40] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 16:04:40] Index already exists. Loading from file.
[Index 16:04:40] Secondary index already exists. Loading from file.
[Index 16:04:40] Index loaded in 0.18 sec.
[Index 16:04:40] Memory consumption: [currentRSS = 513 MB, peakRSS = 513 MB]

[Run 16:04:40] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 16:04:40] Reference genome is assumed to be linear.
[Run 16:04:40] Only one alignment will be reported per mapped read.
[ProcessReads 16:04:40] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 16:04:40] Batch of 81 reads (0 MiB) loaded in 0.00 sec. (24880248 bases)
[ProcessReads 16:04:40] Memory consumption: [currentRSS = 514 MB, peakRSS = 514 MB]
[ProcessReads 16:04:40] Using 4 threads.
[ProcessReads 16:04:42] [CPU time: 3.51 sec, RSS: 535 MB] Read: 81/81 (100.00%) [m: 80, u: 1]                     

In [194]:
bam_file = prepare_sam(args["out_basename"])

[samopen] SAM header is present: 1 sequences.


In [195]:
samfile = pysam.AlignmentFile(bam_file)
reads  = [x for x in samfile.fetch()]
len(reads)

80

In [196]:
def train_read(read_name): 
    file_id, channel_id = get_file_and_channel(read_name)
    file_obj = get_file(channel_id, file_id)      
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    seq = ghmm.EmissionSequence(F, events)
    model.baumWelch(seq)
    

In [197]:
def basecall_read(params):
    read_name, ref_pos = params
    file_id, channel_id = get_file_and_channel(read_name)
    
    file_obj = get_file(channel_id, file_id)
      
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    metrichor_seq = file_obj["fastq"].split("\n")[1]
    ref_seq = "".join([ref[x] for x in ref_pos])
    called_seq = predict(events)
    
    stats = {
        "d_metrichor": int(editdistance.eval(ref_seq, metrichor_seq)),
        "d_caller": int(editdistance.eval(ref_seq, called_seq)), 
        "length": len(ref_seq),
    }

    return (read_name, called_seq, stats)

In [198]:
## one can not access a read in parallel (deadlock for whatever reason)
## therfore prepare input parameters outside of map
input_params = [(read.query_name, read.get_reference_positions()) for read in reads]
read_names = [x[0] for x in input_params]

In [199]:
# for i, read in enumerate(read_names): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(read_names))))
#     train_read(read)

In [ ]:
p = Pool(args["ncores"])

In [ ]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, input_params), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(input_params))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

Prediction: 
done 26.250000%

### Stats

In [ ]:
headers, seqs, stats = zip(*results)

In [ ]:
stats = pandas.DataFrame(list(stats))

In [ ]:
stats = stats.sum(0)

In [ ]:
stats

In [ ]:
print("Relative Performance: {0:5.3f}%".format(stats["d_metrichor"] * 100 /float(stats["d_caller"])))

In [ ]:
# ## Random control
# import random
# seqs = ["".join([random.choice("ACGT") for _ in range(len(read))]) for read in seqs]

In [ ]:
fasta_file_called = "{0}.called.fa".format(args["out_basename"])
with open(fasta_file_called, 'w') as f: 
    for header, seq in zip(headers, seqs): 
        f.write(">" + header + "\n")
        f.write(seq + "\n")

In [ ]:
sam_file_called = "{0}.called.sam".format(args["out_basename"])
graphmap(ref_file, fasta_file_called, sam_file_called, args["ncores"])

In [ ]:
prepare_sam("{0}.called".format(args["out_basename"]))